In [1]:
from bs4 import BeautifulSoup
import requests as req
import pandas as pd
import re
import warnings

warnings.filterwarnings("ignore")

In [2]:
df_cols = {
    "பாடல்": [],
    "படம்": [],
    "வருடம்": [],
    "இசையமைப்பாளர்": [],
    "பாடல் வரிகள்": [],
    "பாடகர்கள்": [],
    "பாடலாசிரியர்": []
}
songs_df = pd.DataFrame(df_cols)
songs_df

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்


In [3]:
final_cols = df_cols.keys()
final_cols

dict_keys(['பாடல்', 'படம்', 'வருடம்', 'இசையமைப்பாளர்', 'பாடல் வரிகள்', 'பாடகர்கள்', 'பாடலாசிரியர்'])

In [4]:
origin = 'https://www.tamilbeatslyrics.com/'
url ='https://www.tamilbeatslyrics.com/category/love-songs'
reg = re.compile(r'[a-zA-Z]')

In [5]:
res = req.get(url)
docs = BeautifulSoup(res.text, 'html.parser')

In [6]:
totalPages = int(docs.findAll("a", class_= "page-numbers")[1].string)
totalPages

6

In [7]:
currentPageNumber = 1

while(currentPageNumber <= totalPages):
    currentURL = url + "/page/" + str(currentPageNumber) ## navigate pagination URL
    currentPageResult = req.get(currentURL)
    currentPageDoc = BeautifulSoup(currentPageResult.text, "html.parser")
    headings = currentPageDoc.findAll("a", attrs={"rel" : "bookmark"})


    ### go through the each page of songs lyrics
    for item in headings:
        songURL = (item.attrs)["href"]
        songResult = req.get(songURL) ## navigate to the song details URL
        songDoc = BeautifulSoup(songResult.text, "html.parser") 
        trs = songDoc.find_all("tr")
        
        songDict = {}

        ### travel thorggh the table to get details of songs
        for tr in trs:
            songDict[tr.th.getText(separator=" ").replace(':', '')] = tr.td.getText(separator=" ")

        lyricsParas = songDoc.find_all("p", class_ = "has-text-align-center")

        ### merge lyrics in single sting
        lyrics = ""
        for lyric in lyricsParas:
            if reg.match(lyric.getText(separator=" ")):
                break
            else:
                lyrics += lyric.getText(separator=" ") + " "
        
        songDict['பாடல் வரிகள்'] = lyrics
        
        # Add to the main lyrics frame
        songs_df = songs_df.append(songDict, ignore_index=True)

    print("Page " + str(currentPageNumber) + " completed ...")
    currentPageNumber += 1

songs_df.shape

Page 1 completed ...
Page 2 completed ...
Page 3 completed ...
Page 4 completed ...
Page 5 completed ...
Page 6 completed ...


(76, 12)

In [8]:
songs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   பாடல்           39 non-null     object
 1   படம்            39 non-null     object
 2   வருடம்          76 non-null     object
 3   இசையமைப்பாளர்   37 non-null     object
 4   பாடல் வரிகள்    76 non-null     object
 5   பாடகர்கள்       27 non-null     object
 6   பாடலாசிரியர்    37 non-null     object
 7   இசை             39 non-null     object
 8   வரிகள்          39 non-null     object
 9   பாடகர்          49 non-null     object
 10  பாடலின் பெயர்   37 non-null     object
 11  படத்தின் பெயர்  37 non-null     object
dtypes: object(12)
memory usage: 7.2+ KB


In [9]:
songs_df['வருடம்'] = songs_df['வருடம்'].fillna('0')
songs_df.fillna('', inplace = True)

In [10]:
songs_df["பாடல்"] = songs_df[["பாடல்", "பாடலின் பெயர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["படம்"] = songs_df[["படம்", "படத்தின் பெயர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["இசையமைப்பாளர்"] = songs_df[["இசையமைப்பாளர்", "இசை"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["பாடகர்கள்"] = songs_df[["பாடகர்கள்", "பாடகர்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df["பாடலாசிரியர்"] = songs_df[["பாடலாசிரியர்", "வரிகள்"]].apply(lambda x: x[0]+x[1], axis =1)
songs_df

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்,இசை,வரிகள்,பாடகர்,பாடலின் பெயர்,படத்தின் பெயர்
0,என்ன இந்த மாற்றமோ,கோரிப்பாளையம்,2010,சபேஷ் முரளி,ஆண்: என்ன இந்த மாற்றமோ என் மனசு வழுக்குதே கண்ண...,"கார்த்திக் , ஸ்ரீமதுமிதா",நா.முத்துக்குமார்,சபேஷ் முரளி,நா.முத்துக்குமார்,"கார்த்திக் , ஸ்ரீமதுமிதா",,
1,குறுமுகில்களை,சீதா ராமம்,2022,விஷால் சந்திரசேகர்,ஆண்: குறுமுகில்களை சிறுமுகைகளில் யார் தூவினார்...,சாய் விக்னேஷ்,மதன் கார்க்கி,விஷால் சந்திரசேகர்,மதன் கார்க்கி,சாய் விக்னேஷ்,,
2,என்னை சாய்த்தாலே,என்றென்றும் புன்னகை,2013,ஹரிஸ் ஜெயராஜ்,ஆண்: என்னை சாய்த்தாலே உயிர் தேய்த்தாலே இனி வாழ...,"ஹரிஹரன், ஸ்ரேயா கோஷல்",தாமரை,ஹரிஸ் ஜெயராஜ்,தாமரை,"ஹரிஹரன், ஸ்ரேயா கோஷல்",,
3,ஒரு பாதி கதவு நீயடி,தாண்டவம்,2012,GV பிரகாஷ் குமாா்,ஆண்: நீ என்பதே நான்தானடி நான் என்பதே நாம்தானடி...,"ஹாிசரண், வந்தனா ஸ்ரீனிவாசன்",நா.முத்துக்குமார்,GV பிரகாஷ் குமாா்,நா.முத்துக்குமார்,"ஹாிசரண், வந்தனா ஸ்ரீனிவாசன்",,
4,இரவாக நீ,இது என்ன மாயம்,2015,GV பிரகாஷ் குமாா்,பெண்: இரவாக நீ நிலவாக நான் உறவாடும் நேரம் சுகம...,"சைந்தவி, GV பிரகாஷ் குமாா்",நா.முத்துக்குமார்,GV பிரகாஷ் குமாா்,நா.முத்துக்குமார்,"சைந்தவி, GV பிரகாஷ் குமாா்",,
...,...,...,...,...,...,...,...,...,...,...,...,...
71,விளம்பர இடைவெளி,இமைக்கா நொடிகள்,2018,ஹிப் ஹாப் தமிழா,ஆண்: ஒளி இல்லா உன் மொழிகள் விடை தேடும் என் விழ...,"கிறிஸ்டோபர் ஸ்டேன்லி , சுதர்ஷன் அசோக், ஹிப் ஹா...",கபிலன் வைரமுத்து,,,,விளம்பர இடைவெளி,இமைக்கா நொடிகள்
72,ஒன்ன விட்ட யாரும் எனக்கில்ல,சீமராஜா,2018,டி.இமான்,பெண்: ஒன்ன விட்ட யாரும் எனக்கில்ல பாரு பாரு என...,"ஸ்ரேயா கோஷல் , சத்ய பிரகாஷ்",யுகபாரதி,,,"ஸ்ரேயா கோஷல் , சத்ய பிரகாஷ்",ஒன்ன விட்ட யாரும் எனக்கில்ல,சீமராஜா
73,உயிர் உருவாத,இரவுக்கு ஆயிரம் கண்கள்,2018,ஷாம்.சி.எஸ்,ஆண்: உயிர் உருவாத உருக்குளைக்காத என்னில் வந்து...,"சத்யா பிரகாஷ், சின்மயி",ஷாம்.சி.எஸ்,,,"சத்யா பிரகாஷ், சின்மயி",உயிர் உருவாத,இரவுக்கு ஆயிரம் கண்கள்
74,ஹை ஆன் லவ்,பியார் பிரேமா காதல்,2018,யுவன் சங்கர் ராஜா,ஏ… பெண்ணே… என் நெஞ்சில் சாய்ந்து சாய்கிறாய் நீ...,சித்ஸ்ரீராம்,நிரஞ்சன் பாரதி,,,சித்ஸ்ரீராம்,ஹை ஆன் லவ்,பியார் பிரேமா காதல்


In [11]:
import numpy as np

songs_df['வருடம்'] = songs_df["வருடம்"].apply(np.int64)
songs_df.dtypes

பாடல்             object
படம்              object
வருடம்             int64
இசையமைப்பாளர்     object
பாடல் வரிகள்      object
பாடகர்கள்         object
பாடலாசிரியர்      object
இசை               object
வரிகள்            object
பாடகர்            object
பாடலின் பெயர்     object
படத்தின் பெயர்    object
dtype: object

In [12]:
songs_df['வருடம்'] = songs_df["வருடம்"].replace(0, np.NAN)
songs_df[final_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   பாடல்          76 non-null     object
 1   படம்           76 non-null     object
 2   வருடம்         76 non-null     int64 
 3   இசையமைப்பாளர்  76 non-null     object
 4   பாடல் வரிகள்   76 non-null     object
 5   பாடகர்கள்      76 non-null     object
 6   பாடலாசிரியர்   76 non-null     object
dtypes: int64(1), object(6)
memory usage: 4.3+ KB


In [13]:
final_df = songs_df[final_cols]
final_df.head(40)

,பாடல்,படம்,வருடம்,இசையமைப்பாளர்,பாடல் வரிகள்,பாடகர்கள்,பாடலாசிரியர்
0,என்ன இந்த மாற்றமோ,கோரிப்பாளையம்,2010,சபேஷ் முரளி,ஆண்: என்ன இந்த மாற்றமோ என் மனசு வழுக்குதே கண்ண...,"கார்த்திக் , ஸ்ரீமதுமிதா",நா.முத்துக்குமார்
1,குறுமுகில்களை,சீதா ராமம்,2022,விஷால் சந்திரசேகர்,ஆண்: குறுமுகில்களை சிறுமுகைகளில் யார் தூவினார்...,சாய் விக்னேஷ்,மதன் கார்க்கி
2,என்னை சாய்த்தாலே,என்றென்றும் புன்னகை,2013,ஹரிஸ் ஜெயராஜ்,ஆண்: என்னை சாய்த்தாலே உயிர் தேய்த்தாலே இனி வாழ...,"ஹரிஹரன், ஸ்ரேயா கோஷல்",தாமரை
3,ஒரு பாதி கதவு நீயடி,தாண்டவம்,2012,GV பிரகாஷ் குமாா்,ஆண்: நீ என்பதே நான்தானடி நான் என்பதே நாம்தானடி...,"ஹாிசரண், வந்தனா ஸ்ரீனிவாசன்",நா.முத்துக்குமார்
4,இரவாக நீ,இது என்ன மாயம்,2015,GV பிரகாஷ் குமாா்,பெண்: இரவாக நீ நிலவாக நான் உறவாடும் நேரம் சுகம...,"சைந்தவி, GV பிரகாஷ் குமாா்",நா.முத்துக்குமார்
5,இருக்கிறாய்,இது என்ன மாயம்,2015,GV பிரகாஷ் குமாா்,பெண்: இருக்கிறாய் இல்லாமலும் இருக்கிறாய் ஆதலால...,"GV பிரகாஷ் குமாா் , ஹரிணி",நா.முத்துக்குமார்
6,காலத்துக்கும் நீ வேணும்,வெந்து தணிந்தது காடு,2022,AR ரஹ்மான்,ஆண்: என்ன தர உன்ன விட நம்பும் ஓர் இடம் இல்ல இன...,"TR சிலம்பரசன், ரக்ஷித்தா சுரேஷ்",தாமரை
7,என்தாரா என்தாரா,திருமணம் என்னும் நிக்காஹ்,2014,ஜிப்ரான்,ஆண்: என்தாரா என்தாரா நீயே என் தாரா என் மனம் பூ...,"ஷதாப் ஃபரிடி, சின்மயி",கார்த்திக் நேதா
8,சின்னஞ்சிறுசுங்க மனசுக்குள் சிலுசிலுன்னு,குங்குமப்பூவும் கொஞ்சும் புறாவும்,2009,யுவன் ஷங்கர் ராஜா,ஆண்: தானனனா தன்னனனா நன்னானனா ஓ ஓ ஓ பெண்: தானனன...,"ஜவ்வாத் அலி, பேலா ஷெண்டே",வாலி
9,நீயே வாழ்க்கை என்பேன்,புகழ்,2016,"விவேக் சிவா, மெர்வின் சாலமன்",நீயே வாழ்க்கை என்பேன் இனி வாழும் நாட்கள் எல்லா...,"அரிஜிட் சிங், மெர்வின் சாலமன்",நா.முத்துக்குமார்


In [14]:
final_df = final_df.replace('', np.nan).dropna().reset_index(drop = True)
final_df['பாடல் வரிகள்'][10]

'பெண்: தனன னானே னனனா தன்னனா தனனா தனனா தனன னானே னனனா தன்னனா தனனா தனனா னானானானா தனனானா தனனானா தானானா ஆண்: ரா ரா ரா ரர ரா ரா ரா ரா ரா ரர ரா ரா ஆண்: அய்யயோ நெஞ்சு அலையுதடி ஆகாயம் இப்போ வளையுதடி என் வீட்டில் மின்னல் ஒளியுதடி என்மேல நிலா பொழியுதடி ஆண்: உன்ன பாா்த்த அந்த நிமிஷம் உறைஞ்சு போச்சு நகரவே இல்ல தின்ன சோறும் சொிக்கவே இல்ல கொலம்புறேன் நானே ஆண்: உன் வாசம் அடிக்கிற காத்து என் கூட நடக்கிறதே என் சேவல் கூவுற சத்தம் உன் பேர கேக்குறதே ஆண்: அய்யயோ நெஞ்சு அலையுதடி ஆகாயம் இப்போ வளையுதடி என் வீட்டில் மின்னல் ஒளியுதடி என்மேல நிலா பொழியுதடி ஆண்: உன்னை தொடும் அனல்காத்து கடக்கையிலே பூங்காத்து குழம்பி தவிக்குதடி என் மனசு பெண்: திருவிழா கடைகளைப் போல திணறுறேன் நான் தானே எதிாில் நீ வரும்போது மிரளுறேன் ஏன்தானோ ஆண்: கண்சிமிட்டும் தீயே என்ன எாிச்சிப்புட்ட நீயே பெண்: ரா ரா ரா ரர ரா ரா ரா ரா ரா ரர ரா ரா பெண்: அய்யயோ நெஞ்சு ஆண்: அலையுதடி பெண்: ஆகாயம் இப்போ ஆண்: வளையுதடி பெண்: என் வீட்டில் மின்னல் ஆண்: ஒளியுதடி பெண்: என்மேல நிலா ஆண்: பொழியுதடி ஆண்: தார ராரரா தார ரார ராரா தார ரார தார ரார ராரா ரா தார ராரரா தார

In [15]:
final_df['உருவகம்'] = pd.Series(np.zeros(final_df.shape[0]))

In [16]:
final_df.to_csv('Tamil Love Songs Lyrics.csv', index=False,  encoding='utf-8')
with open('Tamil Love Songs Lyrics.json', 'w', encoding='utf-8') as fileJSON:
    final_df.to_json(fileJSON, force_ascii=False)